# Basic graphing

## Goal

- PDR vs TxRx-Distance
- Grant breaks of different categories over time

Need a bit more than this but it's a start at least

In [33]:
import re
import os
import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

def create_bins(lower_bound, width, quantity):
    """ create_bins returns an equal-width (distance) partitioning. 
        It returns an ascending list of tuples, representing the intervals.
        A tuple bins[i], i.e. (bins[i][0], bins[i][1])  with i > 0 
        and i < quantity, satisfies the following conditions:
            (1) bins[i][0] + width == bins[i][1]
            (2) bins[i-1][0] + width == bins[i][0] and
                bins[i-1][1] + width == bins[i][1]
    """
    
    bins = []
    for low in range(lower_bound, 
                     lower_bound + quantity*width + 1, width):
        bins.append((low, low+width))
    return bins

In [51]:
pdf = pd.read_csv("/home/brian/results-analysis/data/processed_data/2019-03-23_12:47-mode4/vector.csv",usecols=["node","name", "vectime", "vecvalue"])
pdf2 = pdf[pdf["node"] == 0]

tbs_recived = pdf2[pdf2["name"] == "tbsReceived:vector"]["vecvalue"].values
tbs_received_time = pdf2[pdf2["name"] == "tbsReceived:vector"]["vectime"].values

txrxdistance = pdf2[pdf2["name"] == "txRxDistance:vector"]["vecvalue"].values
txrxdistance_time = pdf2[pdf2["name"] == "txRxDistance:vector"]["vectime"].values

tbs_failbutrec = pdf2[pdf2["name"] == "tbFailedButSCIReceived:vector"]["vecvalue"].values
tbs_failbutrec_time = pdf2[pdf2["name"] == "tbFailedButSCIReceived:vector"]["vectime"].values

tbs_failduesci = pdf2[pdf2["name"] == "tbsFailedDueToNoSCI:vector"]["vecvalue"].values
tbs_failduesci_time = pdf2[pdf2["name"] == "tbsFailedDueToNoSCI:vector"]["vectime"].values

#final_res = (tbs_recived - tbs_failbutrec - tbs_failduesci)/ tbs_recived *100

In [52]:
tbs_recived = np.array(tbs_recived[0].split(" ")).astype("float")
tbs_received_time = np.array(tbs_received_time[0].split(" ")).astype("float")

txrxdistance = np.array(txrxdistance[0].split(" ")).astype("float")
txrxdistance_time = np.array(txrxdistance_time[0].split(" ")).astype("float")

tbs_failbutrec = np.array(tbs_failbutrec[0].split(" ")).astype("float")
tbs_failbutrec_time = np.array(tbs_failbutrec_time[0].split(" ")).astype("float")

tbs_failduesci = np.array(tbs_failduesci[0].split(" ")).astype("float")
tbs_failduesci_time = np.array(tbs_failduesci_time[0].split(" ")).astype("float")

In [56]:
tbs_failbutrec = []
tbs_failduesci = []
dist = []
for inst in tbs_received_time:
   itemindex_failbutrec = np.where(tbs_failbutrec_time==inst)
   tbs_failbutrec.append(len(itemindex_failbutrec[0]))
   itemindex_tbs_failduesci = np.where(tbs_failduesci_time==inst)
   tbs_failduesci.append(len(itemindex_tbs_failduesci[0]))

   itemindex_disttime = np.where(txrxdistance_time==inst)
   dist.append(tbs_txrxdistance[itemindex_disttime[0][0]])
   
    
   
pdf_dict = {}
pdf_dict["tbs_recived"] = tbs_recived
pdf_dict["txrxdistance"] = dist
pdf_dict["tbs_failbutrec"] = np.array(tbs_failbutrec)
pdf_dict["tbs_failduesci"] = np.array(tbs_failduesci)
pdf3 = pd.DataFrame(pdf_dict)
pdf_500m = pdf3[pdf3["txrxdistance"] <= 500]

bins = create_bins(lower_bound=0, width=10, quantity=49)
print (bins)

for interval in bins:
    lower_b = interval[0]
    upper_b = interval[1]
    pdf_temp = pdf_500m[(pdf_500m["txrxdistance"] >= lower_b) & (pdf_500m["txrxdistance"] < upper_b)]
    sum_rec = float(len(pdf_temp["tbs_recived"]))
    pdr = ((sum_rec - pdf_temp["tbs_failbutrec"].sum() - pdf_temp["tbs_failduesci"].sum() ) / sum_rec ) *100
    print (pdr)

print (bins[14])
lower_b = bins[14][0]
upper_b = bins[14][1]
pdf_temp = pdf_500m[(pdf_500m["txrxdistance"] >= lower_b) & (pdf_500m["txrxdistance"] < upper_b)]
sum_rec = float(len(pdf_temp["tbs_recived"]))
print (sum_rec)
print (pdf_temp["tbs_failbutrec"].sum())
print (pdf_temp["tbs_failduesci"].sum() )
pdr = ((sum_rec - pdf_temp["tbs_failbutrec"].sum() - pdf_temp["tbs_failduesci"].sum() ) / sum_rec ) *100
print (pdr)
    
print (pdf_500m.head())

[(0, 10), (10, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100), (100, 110), (110, 120), (120, 130), (130, 140), (140, 150), (150, 160), (160, 170), (170, 180), (180, 190), (190, 200), (200, 210), (210, 220), (220, 230), (230, 240), (240, 250), (250, 260), (260, 270), (270, 280), (280, 290), (290, 300), (300, 310), (310, 320), (320, 330), (330, 340), (340, 350), (350, 360), (360, 370), (370, 380), (380, 390), (390, 400), (400, 410), (410, 420), (420, 430), (430, 440), (440, 450), (450, 460), (460, 470), (470, 480), (480, 490), (490, 500)]
100.0
17.681159420289855
13.910761154855644
54.958677685950406
100.0
100.0
61.6504854368932
54.46927374301676
47.276688453159046
44.55696202531645
30.528375733855185
28.273809523809522
85.18518518518519
18.14946619217082
-69.47674418604652
-41.23711340206185
-13.26530612244898
-26.96078431372549
35.18518518518518
11.904761904761903
8.778625954198473
17.559523809523807
-98.74213836477988
-89.10891089108911
-9.0116279

### My own messing 

In [ ]:
vector_df = pd.read_csv("/home/brian/results-analysis/data/processed_data/2019-03-23_12:47-mode4/vector.csv",usecols=["node","name", "vectime", "vecvalue"])

In [5]:
vector_df.head()

,name,vectime,vecvalue,node
0,posX:vector,1 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,2.6 2.6 17.266391688581 31.114573689193 45.438...,0
1,posY:vector,1 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 1...,0
2,servingCell:vector,1,0,0
3,posX:vector,1 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,2.6 2.6 14.866127466398 27.891350551576 40.864...,1
4,posY:vector,1 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 ...,1


In [18]:
vector_df.vectime.values

array([list(['1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50']),
       list(['1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50']),
       list(['1']), ..., list(['49.927']), list(['49.971']),
       list(['49.971'])], dtype=object)

In [17]:
vector_df.vectime = vector_df.vectime.str.split(" ")
vector_df.vecvalue = vector_df.vecvalue.str.split(" ")
#np.array(tbs_txrxdistance_time[0].split(" ")).astype("float")


In [ ]:
vector_df.vectime = vector_df.vectime.apply(lambda x: np.array(x.astype("float")))
#vector_df.vecvalue = vector_df.vecvalue.apply(lambda x: np.array(x.astype("float")))

In [24]:
vector_df.vectime.values

array([list(['1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50']),
       list(['1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50']),
       list(['1']), ..., list(['49.927']), list(['49.971']),
       list(['49.971'])], dtype=object)

In [ ]:
vector_df = vector_df.pivot_table(index='vectime',columns='name',values='vecvalue', aggfunc='sum')


In [ ]:
vector_df.head()

In [22]:
times = vector_df.vectime.values

In [23]:
times 

array(['1 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50',
       '1 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50',
       '1', ..., '49.927', '49.971', '49.971'], dtype=object)

In [26]:
times.split(" ").astype("float")

AttributeError: 'numpy.ndarray' object has no attribute 'split'

In [19]:
times = vector_df["vectime"].values
times = np.hstack(times)
times = times_other.astype(np.float)

In [34]:
new_times = np.arry
for array in times:
    np.array(times.split(" ")).astype("float"):

AttributeError: 'numpy.ndarray' object has no attribute 'split'

In [20]:
times = np.stack(times)

ValueError: all input arrays must have the same shape

In [22]:
times_other = np.hstack(times)

In [26]:
times_fixed = times_other.astype(np.float)

array([ 1.   ,  1.   ,  2.   , ..., 49.927, 49.971, 49.971])